In [1]:
#!/usr/bin/env python
"""
A script that converts dicom files to an intensity voxel array
The resulting array is 1000 x 1024 in cross section direction and 1014 "tall"
"""

%matplotlib inline
import sys, os, re
import numpy as np
import dicom
from matplotlib import pyplot as plt
from errno import ENOENT

def _regularize_dcm_files(dcm_files):
    # Are we reading a file, a list of files, or files from a directory?
    # 1. Is dcm_files a string?
    if isinstance(dcm_files, str):
        # yes: either a single file or a directory
        if os.path.isdir(dcm_files):
            # create a list of DICOM files from the contents of the directory
            dcm_files = dcm_files.rstrip('/')
            rval = ['{}/{}'.format(dcm_files, fname)
            for fname in os.listdir(dcm_files)
                if re.match(r'.*\.dcm', fname)]
        else:
            # it is it's own file
            rval = [dcm_files]
    else:
        # a list of actual files was passed
        rval = list(dcm_files)
    # IOError
    for fname in rval:
        # Below only works if we are in the same directory as files
        if not os.path.isfile(fname):
            raise IOError(ENOENT, '{} is not a file'.format, fname)
    return rval


In [2]:

def read(dcm_files):
	"""
	Read DICOM formatted file(s).

	Input
	-----
	:dcm_files, str|iterable: DICOM file(s) to process. This understands
		three use cases:

		1. `dcm_files` is a single DICOM file
		2. `dcm_files` is a list of DICOM files
		3. `dcm_files` is a directory containing DICOM files

	Returns
	-------
	Tuple: (3D numpy array of intensities, pixel size)
	"""
    
	dcm_files = _regularize_dcm_files(dcm_files)
	dicom_array = None
	for filename in dcm_files:
		layer = dicom.read_file(filename)
		if dicom_array is None:
			nrows, ncols = int(layer.Rows), int(layer.Columns)
			dicom_array = np.zeros((nrows, ncols, len(dcm_files)))
			pixel_size = 1000*np.array(layer.PixelSpacing).astype(float)
        # Only populate a 2D array for single files sent
        if len(dcm_files)==1:
            dicom_array[:, :, 0] = layer.pixel_array
        # Populate layer-wise for dicom sequences
        else:
            dicom_array[:, :, int(layer.InstanceNumber)-1] = layer.pixel_array
	return dicom_array, pixel_size


In [3]:
# TEST: Try passing an array containing dcm files
dcm_array, px_size = read('P002_B001_D09_dicom')
print dcm_array.shape
print px_size

(1024, 1000, 1009)
[ 4.497  4.497]


In [4]:
# TEST: Try passing a single file name
dcm_array, px_size = read('P002_B001_D09_dicom/P002_B001_D09-0.4X_dicom.dcm0010.dcm')
print dcm_array.shape
print px_size


(1024, 1000, 1)
[ 4.497  4.497]


In [5]:
# TEST: Try passing a list of file names
dcm_array, px_size = read(['P002_B001_D09_dicom/P002_B001_D09-0.4X_dicom.dcm0001.dcm', 
                            'P002_B001_D09_dicom/P002_B001_D09-0.4X_dicom.dcm0002.dcm'])
print dcm_array.shape
print px_size

(1024, 1000, 2)
[ 4.497  4.497]
